<a href="https://colab.research.google.com/github/nooblette/DeepLearning/blob/main/MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets, transforms, utils
import pandas as pd
from torchsummary import summary
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import time
import tqdm

%matplotlib inline

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize
                                transforms.Resize((32, 32))
                                ])
train_dataset = datasets.CIFAR10(root="./data", train = True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root="./data", train = False, transform=transform, download=True)

batch_size = 256

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

device = "cuda" if torch.cuda.is_available() else "cpu"

class Block(nn.Module):
  def __init__(self, in_planes, out_planes, stride=1):
    super(Block, self).__init__()

    self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
    self.bn1 = nn.BatchNorm2d(in_planes)
    self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
    self.bn2 = nn.BatchNorm2d(out_planes)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = F.relu(self.bn2(self.conv2(out)))
    return out

class MobileNet(nn.Module):
  cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, (1024,2), 1024]

  def __init__(self, num_classes=10):
    super(MobileNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(32)
    self.layers = self._make_layers(in_planes=32)
    self.linear = nn.Linear(1024, num_classes)

  def _make_layers(self, in_planes):
    layers = []
    for x in self.cfg:
      out_planes = x if isinstance(x, int) else x[0]
      stride=1 if isinstance(x, int) else x[1]
      layers.append(Block(in_planes, out_planes, stride))

      in_planes = out_planes
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layers(out)
    out = F.avg_pool2d(out, 2)
    out = out.view(out.size(0), -1)
    out = self.linear(out)

    return out

def train():
    shuffleNet = MobileNet().to(device)
    CEloss = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(shuffleNet.parameters(), lr=0.01, momentum=0.9)

    def accuracy(y, label, printable=False):
      with torch.no_grad():
        pred = torch.argmax(y.data, 1)
        correct = (pred==label).sum().item()
        temp_acc = (100*correct / y.shape[0])
        if printable:
          print(f'Accuracy of the network on the test images (batch_size : {y.shape[0]}): {temp_acc}%')

        return temp_acc

    max_epoch = 10
    for epoch in range(max_epoch):
      total_acc = []
      total_loss = []
      start_time = time.time()
      print(f"Epoch {epoch} starts.")

      for data, label in tqdm.tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        y = shuffleNet(data)

        temp_acc = accuracy(y, label, printable = False)
        total_acc.append(temp_acc)

        loss = CEloss(y, label)
        total_loss.append(loss)

        loss.backward()

        optimizer.step()

      print("\n")
      print(f"{epoch} epoch loss : {np.array(total_loss).sum() / len(total_loss)}")
      print(f"{epoch} epoch accuracy : {np.array(total_acc).sum() / len(total_acc)}")
      print(f"{epoch} epoch time : {time.time() - start_time } (s)")
      print("\n")

def test():
    shuffleNet = MobileNet().to(device)
    start_time = time.time()

    total_acc = []
    total_loss = []

    for data, label in tqdm.tqdm(test_loader):
      with torch.no_grad():
        data = data.to(device)
        label = label.to(device)

        y = shuffleNet(data)

        temp_acc = accuracy(y, label)
        total_acc.append(temp_acc)

        loss = CEloss(y, label)
        total_loss.append(loss)

      print("\n")
      print(f"Test loss ; {np.array(total_loss).sum() / len(total_loss)}")
      print(f"Test train accuracy : {np.array(total_acc).sum() / len(total_acc)}")
      print(f"Single epoch Time : {time.time() - start_time} (s)")
      print("\n")

    
train()
test()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 0 starts.


  3%|▎         | 5/196 [00:32<20:43,  6.51s/it]


KeyboardInterrupt: ignored